In [138]:
import pandas as pd
import numpy as np
import altair as alt

In [226]:
events = pd.read_csv("datasets/userTestEvents.csv",delimiter=",")
overview = pd.read_csv("datasets/userTestMonitoring.csv",delimiter=",")
overview = overview[overview["ID_PARTICIPANTE"] != 0 ]
events = events[events["task"] < 6]
# events = events[(events["event"] != "finish_task") & (events["event"] != "close")]

In [141]:
events.columns, overview.columns

(Index(['timestamp', 'TASK_CLICK_NUMBER', 'fullEvent', 'id', 'task', 'event',
        'context', 'subcontext', 'attr', 'station', 'environment', 'element',
        'lx', 'ly', 'vis_id', 'value', 'vis-id', 'time', 'time_shift'],
       dtype='object'),
 Index(['ID_PARTICIPANTE', 'TASK_NUMBER', 'TASK_COMPLETION_TIME',
        'NUMBER_OF_CLICKS_TO_TO_COMPLETE_THE_TASK', 'environment'],
       dtype='object'))

In [123]:
events["event"].unique()

array(['select_station', 'show_cycle', 'back', 'show_kpi', 'show_filters',
       'bottleneck', 'close', 'cancel_filter', 'all', 'select_attr',
       'set_slider', 'line_monitoring', 'start_evaluation', 'close_vis',
       'void_selection'], dtype=object)

In [124]:
overview.head()

,ID_PARTICIPANTE,TASK_NUMBER,TASK_COMPLETION_TIME,NUMBER_OF_CLICKS_TO_TO_COMPLETE_THE_TASK,environment
5,1,1,64.431106,2,ar
6,1,2,32.290960,3,ar
7,1,3,19.741665,2,ar
8,1,4,147.613957,10,ar
9,1,5,103.665919,11,ar


In [125]:
slider = alt.binding_range(min=0, max=100, step=1, name='TASK_COMPLETION_TIME:')
selector = alt.selection_single(name="selector", fields=['TASK_COMPLETION_TIME'],
                                bind=slider, init={'TASK_COMPLETION_TIME': 50})
alt.Chart(overview).mark_bar().encode(
    x="environment",
    y=alt.Y("TASK_COMPLETION_TIME", title="time"),
    color="environment",
    opacity=alt.condition("datum.TASK_COMPLETION_TIME < selector.TASK_COMPLETION_TIME",alt.value(.3),alt.value(1)),
    tooltip="TASK_COMPLETION_TIME"
).properties(width=70, height=70).facet(
    column="ID_PARTICIPANTE",
    row="TASK_NUMBER"
).add_selection(
    selector
)

alt.FacetChart(...)

In [300]:
2303 / 60, 1161 / 60, 2303 / 60 / 8, 1161 / 60 / 8, 

(38.38333333333333, 19.35, 4.797916666666667, 2.41875)

In [126]:
overview.groupby(["environment"]).sum()

,ID_PARTICIPANTE,TASK_NUMBER,TASK_COMPLETION_TIME,NUMBER_OF_CLICKS_TO_TO_COMPLETE_THE_TASK
environment,,,,
ar,180,120,2303.814748,214
tablet,180,120,1161.525000,216


In [127]:
alt.Chart(overview).mark_bar().encode(
    y="count()",
    x=alt.X("TASK_COMPLETION_TIME", bin=alt.BinParams(maxbins=10))
).properties(height=80).facet(
    row="TASK_NUMBER",
    column="environment"
)

alt.FacetChart(...)

In [128]:
base = alt.Chart(overview).mark_bar().encode(
    y="count()",
    x=alt.X("TASK_COMPLETION_TIME", type='quantitative', bin=alt.BinParams(maxbins=30)),
)

base
(base | base.encode(x=alt.X("NUMBER_OF_CLICKS_TO_TO_COMPLETE_THE_TASK", type='quantitative', bin=alt.BinParams(maxbins=30))))

alt.HConcatChart(...)

In [297]:
circles = alt.Chart().mark_circle(opacity=.6).encode(
    y=alt.Y("TASK_COMPLETION_TIME", title="Task completion time"),
    x=alt.X("NUMBER_OF_CLICKS_TO_TO_COMPLETE_THE_TASK", title="Number of clicks/touches"),
)

(alt.Chart().mark_rect().encode(
    y=alt.Y("TASK_COMPLETION_TIME", bin=alt.BinParams(maxbins=10)),
    x=alt.X("NUMBER_OF_CLICKS_TO_TO_COMPLETE_THE_TASK",bin=alt.BinParams(maxbins=5)),
    color=alt.Color("count()", scale=alt.Scale(scheme="teals"))
) + circles.encode(
    fill=alt.value("white"), size=alt.value(50),
) + circles.encode(
    fill=alt.value("black"), size=alt.value(40),
)
).facet(
    column=alt.Column("environment", title=""),
    data=overview[overview["NUMBER_OF_CLICKS_TO_TO_COMPLETE_THE_TASK"] < 20]
)

alt.FacetChart(...)

In [130]:
events[events["environment"] == "tablet"]["task"].unique()

array([2, 3, 4, 5, 1], dtype=int64)

In [177]:
selecao = alt.selection_single(
    name='Selecao',
    fields=['id'],
    init={'id': 1},
    bind=alt.binding_select(options=events["id"].unique())
)

alt.Chart(events).mark_line().encode(
    x=alt.X("time:Q"), #, scale=alt.Scale(domain=[0,100]))
    y=alt.Y("event",sort="x"),
    color="id:N"
).properties(
    width=160,
    height=120
).facet(
    column="task",
    row="environment"
).resolve_scale(
    x="independent"
)
# .transform_filter(
#     "datum.id == Selecao.id"
# ).add_selection(
#     selecao
# )


alt.FacetChart(...)

In [178]:
# (alt.Chart(events).mark_bar().encode(
#     y="sum(time_shift)",
#     x=alt.X("id"),
# ) | alt.Chart(events).mark_bar().encode(
#     y="sum(time)",
#     x=alt.Y("event",sort="-y"),
#     color="task:N"
# ).facet(row="environment"))

alt.HConcatChart(...)

In [143]:
(alt.Chart(events).mark_bar().encode(
    y="sum(time_shift)",
    x=alt.Y("event",sort="-y"),
    color="task:N",
    tooltip="sum(time)"
).facet(column="environment"))

alt.FacetChart(...)

In [305]:
f_events= ["select_station", "set_slider", "show_kpi", "show_filters", "show_cycle"]
# events[events["event"].isin(f_events)]
alt.Chart(events[events["event"].isin(f_events)]).mark_bar().encode(
    x=alt.X("sum(time_shift)", title="Time in seconds"),
    y=alt.Y("environment",sort="y", title=""),
    color=alt.Color("task:N", title="Task"),
    tooltip="sum(time_shift)"
).facet(row=alt.Row("event", title="Events", sort=f_events))

alt.FacetChart(...)

In [210]:
selecao = alt.selection_single(name="selecao", fields=['id'])
# color=alt.condition(selecao,alt.value(),alt.value())
(alt.Chart(events).mark_bar(size=20).encode(
    y="sum(time_shift)",
    x=alt.X("id"),
    color=alt.condition(selecao,alt.value("steelblue"),alt.value("grey"))
).add_selection(
    selecao
) | alt.Chart(events).mark_line().encode(
    x=alt.X("time:Q"),
    y=alt.Y("event",sort="x")
).transform_filter(
    "datum.id == selecao.id"
)).transform_filter(
    "datum.environment == 'ar'"
) & (alt.Chart(events).mark_bar(size=20).encode(
    y="sum(time_shift)",
    x=alt.X("id"),
    color=alt.condition(selecao,alt.value("crimson"),alt.value("grey"))
).add_selection(
    selecao
) | alt.Chart(events).mark_line().encode(
    x=alt.X("time:Q"),
    y=alt.Y("event",sort="x")
).transform_filter(
    "datum.id == selecao.id"
)).transform_filter(
    "datum.environment == 'tablet'"
)

alt.VConcatChart(...)